Install whiteboxtools, in python called [whotbox-workflows](https://www.whiteboxgeo.com/whitebox-workflows-for-python/), rasterio.

Whitebox-workflows(WbW) is a Rust-based geo-processing software combining the Whitebox tools with Python scripting. Easily handle raster, vector, and LiDAR data from a single Python library.

In [ ]:
! pip install whitebox-workflows
! pip install rasterio

In [ ]:
import rasterio
from rasterio.windows import from_bounds

gedtm30='https://s3.opengeohub.org/global/edtm/gedtm_rf_m_30m_s_20060101_20151231_go_epsg.4326.3855_v1.2.tif'

# Bounding box (minx, miny, maxx, maxy)
bbox = (119.25, 21.53, 121.48, 23.23)

with rasterio.open(gedtm30) as src:
    window = from_bounds(*bbox, src.transform)
    out_image = src.read(window=window)
    out_transform = src.window_transform(window)
    out_meta = src.meta.copy()

out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

tmp_dtm_rls_file="gedtm30_tw.tif"

with rasterio.open(tmp_dtm_rls_file, "w", **out_meta) as dest:
    dest.write(out_image)

In [ ]:
# setup environment
import whitebox_workflows as wbw
wbe = wbw.WbEnvironment()
dtm = wbe.read_raster(tmp_dtm_rls_file)

In [ ]:
wbe.max_procs = -1 # allow whitebox to access all the threads

In [ ]:
import time

start_time = time.time()
dtm = wbe.read_raster(tmp_dtm_rls_file)
print(f"read raster--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
dtm = dtm * 0.1 #decimeter to meter (m)
print(f"rescale--- %s seconds ---" % (time.time() - start_time))

tmp_dtm_file = tmp_dtm_rls_file.replace('dtm','filtered.dtm')
start_time = time.time()
dtm = wbe.gaussian_filter(dtm)
print(f"calculate gaussian filter--- %s seconds ---" % (time.time() - start_time))

# fill depression for hydrological analysis
tmp_flled_dtm_file=tmp_dtm_rls_file.replace('dtm','nodepress.dtm')
start_time = time.time()
dtm_no_deps = wbe.fill_depressions(dtm)
wbe.write_raster(dtm_no_deps, tmp_flled_dtm_file, compress=True)#, compress=False) # Compression is good, but it is a bit slower so here we won't use it.
print(f"calculate breach fill depression--- %s seconds ---" % (time.time() - start_time))


# slope for hydrology
tmp_slope_file=tmp_dtm_rls_file.replace('dtm','slope.in.degree')
start_time = time.time()
slope_in_degree = wbe.slope(dtm,units="degrees")
wbe.write_raster(slope_in_degree, tmp_slope_file, compress=True)#, compress=False) # Compression is good, but it is a bit slower so here we won't use it.
print(f"calculate slope--- %s seconds ---" % (time.time() - start_time))

# Specific catchment area(SCA)
tmp_sca_file=tmp_dtm_rls_file.replace('dtm','spec.catch')
start_time = time.time()
sca = wbe.qin_flow_accumulation(dtm_no_deps, out_type='sca')

print(f"calculate speicific catchment area--- %s seconds ---" % (time.time() - start_time))


#Twi
start_time = time.time()
tmp_twi_file=tmp_dtm_rls_file.replace('dtm','twi')
twi = wbe.wetness_index(specific_catchment_area=sca, slope=slope_in_degree)
wbe.write_raster(twi, tmp_twi_file, compress=True) # Compression is good, but it is a bit slower so here we won't use it.
print(f"topographic wetness index--- %s seconds ---" % (time.time() - start_time))
